In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras
import os

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv("/content/Tweets.csv")
df

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14635,569587686496825344,positive,0.3487,NaN,0.0000,American,NaN,KristenReenders,NaN,0,@AmericanAir thank you we got on a different f...,NaN,2015-02-22 12:01:01 -0800,NaN,NaN
14636,569587371693355008,negative,1.0000,Customer Service Issue,1.0000,American,NaN,itsropes,NaN,0,@AmericanAir leaving over 20 minutes Late Flig...,NaN,2015-02-22 11:59:46 -0800,Texas,NaN
14637,569587242672398336,neutral,1.0000,NaN,NaN,American,NaN,sanyabun,NaN,0,@AmericanAir Please bring American Airlines to...,NaN,2015-02-22 11:59:15 -0800,"Nigeria,lagos",NaN
14638,569587188687634433,negative,1.0000,Customer Service Issue,0.6659,American,NaN,SraJackson,NaN,0,"@AmericanAir you have my money, you change my ...",NaN,2015-02-22 11:59:02 -0800,New Jersey,Eastern Time (US & Canada)


In [ ]:
df.columns



Index(['airline_sentiment', 'text'], dtype='object')

In [ ]:
drop_columns = ['tweet_id', 'airline_sentiment_confidence',
       'negativereason', 'negativereason_confidence', 'airline',
       'airline_sentiment_gold', 'name', 'negativereason_gold',
       'retweet_count', 'tweet_coord', 'tweet_created',
       'tweet_location', 'user_timezone']

for column in drop_columns:
    if column not in df.columns:
        print(f"{column} not found in the DataFrame")

tweet_id not found in the DataFrame
airline_sentiment_confidence not found in the DataFrame
negativereason not found in the DataFrame
negativereason_confidence not found in the DataFrame
airline not found in the DataFrame
airline_sentiment_gold not found in the DataFrame
name not found in the DataFrame
negativereason_gold not found in the DataFrame
retweet_count not found in the DataFrame
tweet_coord not found in the DataFrame
tweet_created not found in the DataFrame
tweet_location not found in the DataFrame
user_timezone not found in the DataFrame


In [ ]:
# Remove columns that exist in the DataFrame
drop_columns = [column for column in drop_columns if column in df.columns]

df = df.drop(columns=drop_columns)

In [ ]:
df1 = df
df['text'] = df['text'].str.lower()
df['airline_sentiment']  = df['airline_sentiment'] .str.lower()
df

,airline_sentiment,text
0,neutral,@virginamerica what @dhepburn said.
1,positive,@virginamerica plus you've added commercials t...
2,neutral,@virginamerica i didn't today... must mean i n...
3,negative,@virginamerica it's really aggressive to blast...
4,negative,@virginamerica and it's a really big bad thing...
...,...,...
14635,positive,@americanair thank you we got on a different f...
14636,negative,@americanair leaving over 20 minutes late flig...
14637,neutral,@americanair please bring american airlines to...
14638,negative,"@americanair you have my money, you change my ..."


In [ ]:
df1 = df
df['text'] = df['text'].str.lower()
df['airline_sentiment']  = df['airline_sentiment'] .str.lower()
df

,airline_sentiment,text
0,neutral,@virginamerica what @dhepburn said.
1,positive,@virginamerica plus you've added commercials t...
2,neutral,@virginamerica i didn't today... must mean i n...
3,negative,@virginamerica it's really aggressive to blast...
4,negative,@virginamerica and it's a really big bad thing...
...,...,...
14635,positive,@americanair thank you we got on a different f...
14636,negative,@americanair leaving over 20 minutes late flig...
14637,neutral,@americanair please bring american airlines to...
14638,negative,"@americanair you have my money, you change my ..."


In [ ]:
# Example contractions dictionary
contractions_dict = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "couldn't": "could not",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'll": "he will",
  "here's": "here is",
  "how's": "how is",
  "i'd": "i would",
  "i'll": "i will",
  "i'm": "i am",
  "isn't": "is not",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mightn't": "might not",
  "mustn't": "must not",
  "shan't": "shall not",
  "she'd": "she would",
  "she'll": "she will",
  "that's": "that is",
  "there's": "there is",
  "they'd": "they would",
  "they're": "they are",
  "they've": "they have",
  "this's": "this is",
  "wasn't": "was not",
  "we'd": "we would",
  "we'll": "we will",
  "weren't": "were not",
  "what's": "what is",
  "when's": "when is",
  "where's": "where is",
  "who's": "who is",
  "why's": "why is",
  "won't": "will not",
  "you'd": "you would",
  "you'll": "you will",
  "you're": "you are",
    "you've": "you have"
}

def expand_contractions(text, contractions_dict):
  """
  Expands contractions in text using a dictionary.

  Args:
    text: The text to be processed.
    contractions_dict: A dictionary of contractions and their expansions.

  Returns:
    The expanded text.
  """
  expanded_text = []
  for word in text.split():
    if word.lower() in contractions_dict:
      expanded_text.append(contractions_dict[word.lower()])
    else:
      expanded_text.append(word)
  return " ".join(expanded_text)
df['text'] = df['text'].apply(expand_contractions, args=(contractions_dict,))
df

,airline_sentiment,text
0,neutral,@virginamerica what @dhepburn said.
1,positive,@virginamerica plus you have added commercials...
2,neutral,@virginamerica i did not today... must mean i ...
3,negative,@virginamerica it is really aggressive to blas...
4,negative,@virginamerica and it is a really big bad thin...
...,...,...
14635,positive,@americanair thank you we got on a different f...
14636,negative,@americanair leaving over 20 minutes late flig...
14637,neutral,@americanair please bring american airlines to...
14638,negative,"@americanair you have my money, you change my ..."


In [ ]:
!pip install nltk
from nltk import download
download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df['text_tokened'] = df['text'].apply(word_tokenize)
df

,airline_sentiment,text,text_tokened
0,neutral,@virginamerica what @dhepburn said.,"[@, virginamerica, what, @, dhepburn, said, .]"
1,positive,@virginamerica plus you have added commercials...,"[@, virginamerica, plus, you, have, added, com..."
2,neutral,@virginamerica i did not today... must mean i ...,"[@, virginamerica, i, did, not, today, ..., mu..."
3,negative,@virginamerica it is really aggressive to blas...,"[@, virginamerica, it, is, really, aggressive,..."
4,negative,@virginamerica and it is a really big bad thin...,"[@, virginamerica, and, it, is, a, really, big..."
...,...,...,...
14635,positive,@americanair thank you we got on a different f...,"[@, americanair, thank, you, we, got, on, a, d..."
14636,negative,@americanair leaving over 20 minutes late flig...,"[@, americanair, leaving, over, 20, minutes, l..."
14637,neutral,@americanair please bring american airlines to...,"[@, americanair, please, bring, american, airl..."
14638,negative,"@americanair you have my money, you change my ...","[@, americanair, you, have, my, money, ,, you,..."


In [ ]:
def replace_with_person_or_airline(text_tokens, airlines_dict):
    for i in range(len(text_tokens)):
        if text_tokens[i] == '@' and text_tokens[i+1] in airlines_dict:
            text_tokens[i+1] = "airline"
            i += 2  # Skip next word as well
        elif text_tokens[i] == '@':
            text_tokens[i+1] = "person"
            i += 2  # Skip next word as well
    return text_tokens
df['cleaned'] = df['text_tokened'].apply(replace_with_person_or_airline,args=(airlines,))
df['cleaned']
df['cleaned']= df['cleaned'].apply(lambda x: [y for y in x if y != '@'])
df['cleaned']

In [ ]:
from keras.preprocessing.text import Tokenizer

# Create a tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['cleaned'])

# Convert text to sequences of numbers
sequences = tokenizer.texts_to_sequences(df['cleaned'])
vocab_size = len(tokenizer.index_word)
vocab_size

In [ ]:
from keras.utils import pad_sequences
sequences = pad_sequences(sequences,padding='pre')
sequences.shape

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
# Initialize LabelEncoder
le = LabelEncoder()

# Apply LabelEncoder to 'airline_sentiment'
df['airline_sentiment_labelencoded'] = le.fit_transform(df['airline_sentiment'])
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(sequences, df['airline_sentiment_labelencoded'], test_size=0.2, random_state=42)

In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Dense, Bidirectional, SimpleRNN,GRU,LSTM,Dropout

model = Sequential([
    Embedding(input_dim=16465,output_dim=2,input_length=46),
    Bidirectional(SimpleRNN(53)),
    Dropout(0.5),
    Dense(12,activation='relu'),
    Dense(3,activation='softmax')
])

#compiling model
model.compile(optimizer="rmsprop",loss='sparse_categorical_crossentropy',metrics=['accuracy'])

model.summary()

In [ ]:
# Predict on the test data
predictions = model.predict(X_test[-200:])
# Now evaluate the model using metrics like accuracy, precision, recall, etc.
from sklearn.metrics import accuracy_score, classification_report

# Convert predictions to class labels (assuming predictions are in one-hot encoded format)
predicted_labels = predictions.argmax(axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_test[-200:], predicted_labels)
print(f"Accuracy: {accuracy}")